In [1]:
import pandas as pd
import os

# List all your normalized CSV files here
normalized_files = [
    "just_blr_normalized.csv",
    "mainland_china_normalized.csv",
    "paakashala_normalized.csv",
    "punjab_grill_normalized.csv",
    "rameshwaram_cafe_normalized.csv"
]

# Initialize an empty list to hold all DataFrames
combined_dataframes = []

# Load each file and add to list
for file in normalized_files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        combined_dataframes.append(df)
    else:
        print(f"File not found: {file}")

# Concatenate all DataFrames
combined_df = pd.concat(combined_dataframes, ignore_index=True)

# Save combined data to a new CSV file
combined_df.to_csv("normalized_restaurant_data.csv", index=False)
print("Combined data saved to 'normalized_restaurant_data.csv'")


Combined data saved to 'normalized_restaurant_data.csv'


In [2]:
import pandas as pd

# Loading normalized restaurant dataset
df = pd.read_csv("normalized_restaurant_data.csv")


# Clean Section and Item Names
df["section"] = df["section"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip().str.title()
df["item"] = df["item"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip().str.title()


# Feature Tagging: vegetarian, non-veg, spicy
def tag_features(row):
    item = row['item'].lower()
    desc = str(row['description']).lower()
    tags = set()
    if any(k in item or k in desc for k in ["veg", "vegetarian", "paneer", "dal", "chole", "palak"]):
        tags.add("vegetarian")
    if any(k in item or k in desc for k in ["chicken", "fish", "egg", "mutton", "lamb", "prawn"]):
        tags.add("non-vegetarian")
    if any(k in item or k in desc for k in ["spicy", "chilli", "hot", "pepper"]):
        tags.add("spicy")
    return list(tags)

df["features"] = df.apply(tag_features, axis=1)


# 3. Fill Missing Descriptions
df["description"] = df["description"].fillna("").replace("nan", "")
df["description"] = df.apply(
    lambda row: f"{row['item']} is a dish from {row['restaurant']} listed under {row['section']}."
    if row["description"].strip() == "" else row["description"], axis=1)


# Fill Missing Hours
df["hours"] = df["hours"].fillna("Hours not available")


# Generate Combined Field for Embedding
df["combined_text"] = df.apply(
    lambda row: f"""Restaurant: {row['restaurant']}
Section: {row['section']}
Item: {row['item']}
Price: {row['price']}
Description: {row['description']}
Features: {', '.join(row['features'])}
Location: {row['location']}
Contact: {row['contact']}
Hours: {row['hours']}""",
    axis=1
)


# Save Enhanced Data
df.to_csv("enhanced_restaurant_data.csv", index=False)
print("Dataset enhanced and saved as 'enhanced_restaurant_data.csv'")


Dataset enhanced and saved as 'enhanced_restaurant_data.csv'


In [3]:
# Install required packages (Colab or local)
!pip install -q gradio sentence-transformers transformers scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [4]:
import requests

def rephrase_query(query):
    prompt = f"Rewrite this user question to be more precise for a multi-restaurant menu assistant:\n\n{query}\n\nRephrased:"
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "mistral", "prompt": prompt, "stream": False}
        )
        refined = response.json()["response"].strip()
        return refined
    except Exception as e:
        print("Rephrasing failed:", e)
        return query  # fallback to original


In [5]:


# Imports
import gradio as gr
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# Load enhanced data
df = pd.read_csv("enhanced_restaurant_data.csv")
text_data = df["combined_text"].tolist()

# Load models
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
generator = pipeline("text2text-generation", model="google/flan-t5-small")

# Compute embeddings
embeddings = embedding_model.encode(text_data, show_progress_bar=True)

# Chatbot logic
def respond(user_query, history):
    if not user_query.strip():
        warning = "Please enter a valid question."
        history.append((user_query, warning))
        return history, history, ""

    refined_query = rephrase_query(user_query)
    print("📝 Rephrased Query:", refined_query)  # Optional for debugging
    query_embedding = embedding_model.encode([refined_query])


    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_k = 4
    top_indices = similarities.argsort()[-top_k:][::-1]
    context = "\n\n".join([f"Info {i+1}:\n{text_data[i]}" for i in top_indices])

    prompt = f"""
You are a helpful assistant trained on the menus of five restaurants: Just BLR, Paakashala, Punjab Grill, Mainland China, and The Rameshwaram Café.

Use the following restaurant menu information to answer customer questions clearly and factually.

Menu Info:
{context}

Question: {refined_query}
Answer:"""


    try:
        response = generator(prompt, max_length=300, temperature=0.7, do_sample=False)
        reply = response[0]['generated_text']
    except Exception as e:
        reply = f"⚠️ Sorry, something went wrong: {str(e)}"

    history.append((user_query, reply))
    return history, history, ""  # Clear input

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🍴 Welcome to the Multi-Restaurant RAG Chatbot!")
    gr.Markdown("""
🤖 I'm trained on menus from 5 restaurants: **Just BLR**, **Paakashala**, **Punjab Grill**, **Mainland China**, and **The Rameshwaram Café**.
💡 I can help you explore dishes, compare prices, check timings, and more.

**Try asking:**
- *Which places serve vegetarian starters under ₹300?*
- *What spicy dishes are served at Punjab Grill?*
- *When is Just BLR open?*
""")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Ask a menu question", placeholder="e.g., Show me veg thali options.")
    state = gr.State([])

    msg.submit(respond, [msg, state], [chatbot, state, msg])

demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

<ipython-input-5-994ff9e7a3f2>:70: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7541de9e00c238ec1e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
